<a href="https://colab.research.google.com/github/jpbeaud/OpenClassProjet/blob/main/client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import time

Traitement du fichier DH

In [3]:


def string_to_number(s):
  return int(''.join(str(ord(char)) for char in s))



url = "https://raw.githubusercontent.com/jpbeaud/fichiers/refs/heads/main/prof_DH.csv"
debut = time.time()
print("Début à", debut)



fichier_DH = pd.read_csv(url, sep=",", dtype=str,skipinitialspace=True)

print("Fichier initiale")
print(fichier_DH.head())
print(fichier_DH.shape)
print("____________________________________________")
fichier_DH = fichier_DH.replace("", np.nan)
fichier_DH = fichier_DH.fillna(0)

fichier_DH = fichier_DH.astype(str)
col_ref = ['id_personne']
references_DH = fichier_DH[col_ref]



fichier_DH.sort_values(by='id_personne')

fichier_DH2 = fichier_DH
fichier_DH2 = fichier_DH2.drop('id_personne', axis='columns')

fichier_DH2= fichier_DH2.astype(str)
fichier_DH2 = fichier_DH2.map(string_to_number)
print ("fichier DH2 format :")
print(fichier_DH2.head())


#fichier_DH.set_index('id_personne', inplace=True) # inplace=True --> on ne crée pas un nouveau DF



Début à 1736969582.623789
Fichier initiale
  id_personne cosop_gmf code_pcs_marque csp_gmf  date_debut
0  5000000525        31              51      75  2017-07-04
1  5000000555        50              51      95  2016-02-08
2  5000001435        76              51      75  2020-09-24
3  5000001975        41              80      75  2005-02-22
4  5000002635        93             NaN      91  2015-06-17
(37709, 5)
____________________________________________
fichier DH2 format :
   cosop_gmf  code_pcs_marque  csp_gmf            date_debut
0       5149             5349     5553  50484955454855454852
1       5348             5349     5753  50484954454850454856
2       5554             5349     5553  50485048454857455052
3       5249             5648     5553  50484853454850455050
4       5751               48     5749  50484953454854454955


Traitement fichier SIO

In [5]:
url = "https://raw.githubusercontent.com/jpbeaud/fichiers/refs/heads/main/prof_SIO.csv"

fichier_SIO = pd.read_csv(url, sep=",", dtype=str,skipinitialspace=True)
col_ref = ['id_personne']

references_SIO= fichier_SIO[col_ref]
ref_cli = references_SIO.to_numpy()

fichier_SIO = fichier_SIO.replace("", np.nan)
fichier_SIO = fichier_SIO.fillna(0)



fichier_SIO.sort_values(by='id_personne')
fichier_SIO = fichier_SIO.astype(str)
fichier_SIO = fichier_SIO.map(string_to_number)


fichier_SIO = fichier_SIO.drop('id_personne',axis='columns')
print ("fichier SIO format :")
print(fichier_SIO.head())
# X = fichier_DH.to_numpy()
X = fichier_DH2
#X.set_index('id_personne', inplace=True) # inplace=True --> on ne crée pas un nouveau DF

# print(X.dtype)
X = X.apply(pd.to_numeric)

import matplotlib.pyplot as plt
# pltcatter(X[:, 3], X[:, 4])


#plt.scatter(x="pcs" , y="cosop", c = 'r', data=X)
#plt.show()

from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

# création du jeu de test

X = StandardScaler().fit_transform(X)
print ("fichier DH2 après scaler :")
print(X[:5])

from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
min_groupe = 2
db = DBSCAN(eps=0.000000000000001,min_samples=min_groupe).fit(X)
labels_DH = db.labels_

n_clusters_ = len(set(labels_DH)) - (1 if -1 in labels_DH else 0)
n_noise_ = list(labels_DH).count(-1)

print("Estimated number of clusters DH: %d" % n_clusters_)
print("Estimated number of noise points DH: %d" % n_noise_)


Y = fichier_SIO


Y = StandardScaler().fit_transform(Y)
print ("fichier SIO après scaler :")
print(Y[:5])


# Y = Y.apply(pd.to_numeric)
db2 = DBSCAN(eps=0.00000000000001, min_samples=min_groupe).fit(Y)
labels_SIO = db2.labels_



n_clusters_ = len(set(labels_SIO)) - (1 if -1 in labels_SIO else 0)
n_noise_ = list(labels_SIO).count(-1)

print("Estimated number of clusters SIO: %d" % n_clusters_)
print("Estimated number of noise points SIO: %d" % n_noise_)
print("____________Résumé____________")
difference = labels_DH - labels_SIO
print("Données en écarts = ", difference)
print(labels_DH)
print(labels_SIO)
n_DH = len(labels_DH)
n_SIO=len(labels_SIO)
n_dif = len(difference)
print("Longueur dif = ", n_dif)
fin = False
i=-1

nb_ko = 0
while (i < n_dif - 1):
    i = i + 1
    if (difference[i] != 0):
        print("i=",i)
        print("différence enregistrement ", str(i), " DH=", labels_DH[i], "// SIO=",labels_SIO[i], "// ref=", ref_cli[i,:])
        #print("enr DH=", fichier_DH.loc[i,:])
        #print("enr SIO=",fichier_SIO.loc[i,:])
        nb_ko = nb_ko + 1
        X = np.delete(X,i,0)
        Y = np.delete(Y,i,0)
        ref_cli = np.delete(ref_cli,i,0)
        db = DBSCAN(eps=0.000000000000001,min_samples=min_groupe).fit(X)
        labels_DH = db.labels_
        db2 = DBSCAN(eps=0.00000000000001, min_samples=min_groupe).fit(Y)
        labels_SIO = db2.labels_
        difference = labels_DH - labels_SIO
        n_dif = n_dif - 1


lg = len(difference)
#nb_ano = 0
#for i in range(lg):
#    if difference[i] !=0:
#        nb_ano = nb_ano + 1
print("ANOMALIES = ", nb_ko)
#  3117773703

fin_exec = time.time()
temps = fin_exec - debut
print(f'Temps d\'exécution : {temps:.2}ms')

fichier SIO format :
    cosop_gmf  code_pcs_marque  csp_gmf            date_debut
0  5149323232             5349     5553  50484955454855454852
1  5348323232             5349     5753  50484954454850454856
2  5554323232             5349     5553  50485048454857455052
3  5249323232             5648     5553  50484853454850455050
4  5751323232               48     5749  50484953454854454955
fichier DH2 après scaler :
[[-0.18282332  0.74826588  0.41124689  0.7785704 ]
 [-0.18068299  0.74826588  0.50540087  0.77857036]
 [-0.17846736  0.74826588  0.41124689  0.77857421]
 [-0.18174778  0.86328833  0.41124689  0.77856623]
 [-0.17634853 -1.29097833  0.50351779  0.77857032]]
Estimated number of clusters DH: 4317
Estimated number of noise points DH: 11389
fichier SIO après scaler :
[[ 0.33963198  0.74826588  0.41124689  0.82434956]
 [ 0.42757987  0.74826588  0.50540087  0.82434851]
 [ 0.5186214   0.74826588  0.41124689  0.82444676]
 [ 0.3838269   0.86328833  0.41124689  0.82424294]
 [ 0.6056853

KeyboardInterrupt: 